## Some comparisons of filtering and smoothing (can be with or without the viewer)

### Hint for your task 2)
#### You must implement your filtering / smoothing in the HMMFilter / HMMSmoother classes in the file Filters. For the filter, this means also that you can use the visualisation through the Localizer, but you can call filter / smooth also directly from here, without the Localiser. Note also, that you must make use of the given models and the robot simulator in RobotSim!


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

from models import *
import Filters

class Localizer:
    def __init__(self, sm, uniformF, skip_steps=False):
        self.__sm = sm
        self.__skip_steps = skip_steps

        self.__tm = TransitionModel(self.__sm)
        if uniformF == True:
            self.__om = ObservationModel_UF.ObservationModelUF(self.__sm)
        else:
            self.__om = ObservationModel_NUF.ObservationModel(self.__sm)

        # change in initialise in case you want to start out with something else...
        self.initialise()

    # retrieve the transition model that we are currently working with
    def get_transition_model(self) -> np.array:
        return self.__tm

    # retrieve the observation model that we are currently working with
    def get_observation_model(self) -> np.array:
        return self.__om

    # the current true pose (x, h, h) that should be kept in the local variable __trueState
    def get_current_true_pose(self) -> (int, int, int):
        x, y, h = self.__sm.state_to_pose(self.__trueState)
        return x, y, h

    # the current probability distribution over all states
    def get_current_f_vector(self) -> np.array(float):
        return self.__probs

    # the current sensor reading (as position in the grid). "Nothing" is expressed as None
    def get_current_reading(self) -> (int, int):
        ret = None
        if self.__sense != None:
            ret = self.__sm.reading_to_position(self.__sense)
        return ret;

    # get the currently most likely position, based on single most probable pose
    def most_likely_position(self) -> (int, int):
        return self.__estimate

    # (re-)initialise for a new run without change of size
    def initialise(self):
        self.__trueState = random.randint(0, self.__sm.get_num_of_states() - 1)
        self.__sense = None
        self.__probs = np.ones(self.__sm.get_num_of_states()) / (self.__sm.get_num_of_states())
        self.__estimate = self.__sm.state_to_position(np.argmax(self.__probs))

        # simple evaluation measures that go into the visualisation, add more if you like!

        self.__rs = RobotSim( self.__trueState, self.__sm)
        self.__HMM = Filters.HMMFilter( self.__probs, self.__tm, self.__om, self.__sm)
        self.__HMM_smooth = Filters.HMMSmoother(self.__HMM)
        self.__HMM_forward_backward = Filters.HMMForwardBackward(self.__HMM, self.__HMM_smooth)

        # Keep track of last 5 sensor readings
        self.sensor_history = []


    #
    #  The update cycle:
    #  - robot moves one step, generates new state / pose
    #  - sensor produces one reading based on the true state / pose
    #  - filtering approach produces new probability distribution based on
    #  sensor reading, transition and sensor models
    #
    #  Reports back to the caller (viewer):
    #  Return
    #  - true if sensor reading was not "nothing", else false,
    #  - AND the three values for the (new) true pose (x, y, h),
    #  - AND the two values for the (current) sensor reading (if not "nothing")
    #  - AND the error made in this step
    #  - AND the new probability distribution
    #
    def update(self) -> (bool, int, int, int, int, int, int, int, int, np.array(1)) :
        # Move robot and get sensor reading
        self.__trueState = self.__rs.move_once(self.__tm)
        self.__sense = self.__rs.sense_in_current_state(self.__om)

        # Update sensor history
        self.sensor_history.append(self.__sense)
        if len(self.sensor_history) > 5:
            self.sensor_history.pop(0)

        # Forward filtering
        self.__probs = self.__HMM.filter(self.__sense)

        # If we have 5 sensor readings, we can do smoothing
        if(len(self.sensor_history) == 5):
            self.__probs = self.__HMM_smooth.smooth(self.sensor_history, self.__probs)

        # calculate position estimates
        fPositions = self.__probs.copy()
        for state in range(0, self.__sm.get_num_of_states(), 4) :
            fPositions[state:state+4] = sum(fPositions[state:state+4])
        self.__estimate = self.__sm.state_to_position(np.argmax(fPositions))
        
        # return values
        ret = False
        tsX, tsY, tsH = self.__sm.state_to_pose(self.__trueState)
        srX = -1
        srY = -1
        if self.__sense is not None:
            srX, srY = self.__sm.reading_to_position(self.__sense)
            ret = True

        eX, eY = self.__estimate

        # calculate error (skip if sensor reading is "nothing" and skip_steps is True)
        if self.__skip_steps and self.__sense is None:
            error = -1
        else:
            error = abs(tsX-eX)+abs(tsY-eY)

        return ret, tsX, tsY, tsH, srX, srY, eX, eY, error, self.__probs

    # update in case the true pose (trajectory) is known and fed into the Localizer from an external source
    def updateWTruePose(self, trueState) -> (bool, int, int, int, int, int, int, int, int, np.array(1)) :
        self.__trueState = trueState
        self.__sense = self.__rs.sense_in_current_state()
        self.__probs = self.__HMM.filter( self.__sense)

        fPositions = self.__probs.copy()

        for state in range(0, self.__sm.get_num_of_states(), 4) :
            fPositions[state:state+4] = sum(fPositions[state:state+4])


        self.__estimate = self.__sm.state_to_position(np.argmax(fPositions))

        ret = False  # in case the sensor reading is "nothing" this is kept...
        tsX, tsY, tsH = self.__sm.state_to_pose(self.__trueState)
        srX = -1
        srY = -1
        if self.__sense != None:
            srX, srY = self.__sm.reading_to_position(self.__sense)
            ret = True

        eX, eY = self.__estimate

        error = abs(tsX-eX)+abs(tsY-eY)


        return ret, tsX, tsY, tsH, srX, srY, eX, eY, error, fPositions

